In [2]:
from modules.result_loader import RelevantDataProcessor, RelevantDataStatistics
from modules.default_config import get_default_config
import numpy as np

/home/richard/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [3]:
config = get_default_config()
PROCESSED_RESULTS_FOLDER = "./results"
processor = RelevantDataProcessor(PROCESSED_RESULTS_FOLDER, config.experiments)
#processor.extract_and_save_relevant_data("graphdata.npy") # Reads 15 GB of data

### relevant data dict:
### all_stats:
###     (centroid name, function name, dimensionality)
###         count_at_min_value
###         total_min_value
###         avg_count
###         avg_min_value
###         best_vals_avg
###         best_vals_std


Config prepared, n_experiments=2550


In [4]:
data = processor.load_prepared_relevant_data("graphdata.npy")
stats = RelevantDataStatistics(data)
#stats.generate_convergence_curves_for_fc_x_dim(save="./results/graphics/convergence")

In [15]:
from modules.centroids import ALL_CENTROIDS
def normalized_ranking(data, fc_weights=None, dim_weights=None, evalfc=None):
    if fc_weights is None:
        fc_weights = {x: 1 for x in data.keys()}
    if dim_weights is None:
        dim_weights = {10: 1, 50: 1, 100: 1}

    results = {x.__qualname__: 0 for x in ALL_CENTROIDS}
    step = 1 / (len(ALL_CENTROIDS) - 1)
    for f in data.keys():
        for dim in data[f].keys():
            i = 0
            for x in data[f][dim]:
                results[x[1]] += fc_weights[f] * dim_weights[dim] * i
                i += step
    return results

In [16]:
s = stats.get_centroids_for_each_function_sorted_by("avg_min_value")
normalized_ranking(s)

{'vanila_centroid': 51.75,
 'mean_centroid': 37.75,
 'median_centroid': 85.0,
 'interquartile_centroid': 50.5,
 'windsor_centroid': 30.0}